In [28]:
import torch
from torch import nn
import pickle
import numpy as np
from torch.autograd import Variable
import os
os.chdir('..')
from models.retain_time import RETAIN

In [17]:
# in_list = []
# with open('data/preprocessed/H26/list_data_2014.pckl','rb') as f:
#     in_list.extend(pickle.load(f))
# with open('data/preprocessed/H26/list_data_2015.pckl','rb') as f:
#     in_list.extend(pickle.load(f))
patient_list = [x[0][0] for x in in_list]

In [33]:
model = RETAIN(hidden_size=128,input_size=128,num_classes=1,decay_ver=2,bidirectional=True,cuda_flag=True)
model.load_state_dict(torch.load('experiments/H26/saved_weights/time_2_best_cuda.pckl'))
model.release = True

In [34]:
model.cuda()

RETAIN(
  (RNN1): TimeLSTM(
    (W_all): Linear(in_features=128, out_features=512, bias=True)
    (U_all): Linear(in_features=128, out_features=512, bias=True)
    (W_d): Linear(in_features=128, out_features=128, bias=True)
  )
  (RNN2): TimeLSTM(
    (W_all): Linear(in_features=128, out_features=512, bias=True)
    (U_all): Linear(in_features=128, out_features=512, bias=True)
    (W_d): Linear(in_features=128, out_features=128, bias=True)
  )
  (RNN1b): TimeLSTM(
    (W_all): Linear(in_features=128, out_features=512, bias=True)
    (U_all): Linear(in_features=128, out_features=512, bias=True)
    (W_d): Linear(in_features=128, out_features=128, bias=True)
  )
  (RNN2b): TimeLSTM(
    (W_all): Linear(in_features=128, out_features=512, bias=True)
    (U_all): Linear(in_features=128, out_features=512, bias=True)
    (W_d): Linear(in_features=128, out_features=128, bias=True)
  )
  (wa): Linear(in_features=256, out_features=1, bias=False)
  (Wb): Linear(in_features=256, out_features=128, 

In [35]:
np_list = [] # list to store np arrays for each sample
count_list = [] # list to store how many times each code appears for each sample
len_list = [] # list to store the length of each sample
for i,patient in enumerate(patient_list):
    if i%100==0:
        print(i)
    score_arr = np.zeros((1400),dtype=float)
    count_arr = np.zeros((1400),dtype=int)
    _,dates_,inputs_,_ = zip(*patient)
    date_list = [list(dates_)]
    input_list = [list(inputs_)]
    inputs = model.list_to_tensor(input_list)
    dates = Variable(torch.Tensor(date_list),requires_grad=False)
    dates = dates.cuda()
    outputs = model(inputs,dates)
    for v,codes in enumerate(inputs_):
        for c in codes:
            s = model.interpret(u=0,v=v,i=c,o=0)
            score_arr[c]+=s.data[0]
            count_arr[c]+=1
    np_list.append(score_arr)
    count_list.append(count_arr)
    len_list.append(len(inputs_))
np_list = np.array(np_list)
count_list = np.array(count_list,dtype=int)
with open('data/postprocessed/H26_scores.pckl','wb') as f:
    pickle.dump((np_list,count_list,len_list),f)
#     input_list.append(list(inputs_))
# inputs = model.list_to_tensor(input_list)
# dates = Variable(torch.Tensor(date_list), requires_grad=False)
# targets = Variable(torch.Tensor(np.array(y,dtype=int)))
# outputs = model(inputs,dates)    

0


KeyboardInterrupt: 

In [40]:
len(inputs_)

44